# MED-PC Extracting the Recording Data and Metadata

## Importing the Python Libraries

In [1]:
import sys
import glob
from collections import defaultdict
import os
import datetime
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from moviepy.editor import *
from datetime import datetime
from IPython.display import Video

In [2]:
from medpc2excel.medpc_read import medpc_read

In [3]:
# setting path
sys.path.append('../../src')

In [4]:
# All the libraries that were created for this repository
import extract.dataframe
import processing.tone
import extract.metadata

## Getting the Metadata from all the MED-PC Recording Files

In [5]:
VideoFileClip??

- Getting all the file paths of the recording files(that happen to all end in `.txt`

# NOTE: The following path must be changed to the directory where your MED-PC recording files are located, if they are not in the specied folder

In [6]:
all_med_pc_file = glob.glob("./data/med_pc/*.txt")

- Use this instead if you're using you're own data

In [7]:
all_med_pc_file[:10]

['./data/med_pc/2022-05-20_10h24m_Subject 2.1 v 1.3.txt',
 './data/med_pc/2022-05-19_09h09m_Subject 2.3v1.4.txt']

- Example of what the MED-PC Recording file looks like

In [8]:
with open(all_med_pc_file[0]) as f:
    lines = f.readlines()
    for line in lines[:20]:
        print(line)

File: C:\MED-PC\Data\2022-05-20_10h24m_Subject 2.1 v 1.3.txt







Start Date: 05/20/22

End Date: 05/20/22

Subject: 2.1 v 1.3

Experiment: Pilot of Pilot

Group: Cage 2

Box: 4

Start Time: 10:24:10

End Time: 11:55:14

MSN: C57_reward

A:    1599.000

D:    8000.000

F:    2000.000

G:       0.000

H:       0.000

I:       0.000

L:       0.000



- We will be extracting the first 10 or so lines that look like:

```
File: C:\MED-PC\Data\2022-05-06_12h59m_Subject 3.4 (2).txt

Start Date: 05/06/22

End Date: 05/06/22

Subject: 3.4 (2)

Experiment: Pilot of Pilot

Group: Cage 4

Box: 1

Start Time: 12:59:58

End Time: 14:02:38

MSN: levelNP_CS_reward_laserepochON1st_noshock
```
    
- We will just find all the lines that start with "File", "Start Date", "End Date", "Subject", "Experiment", "Group", "Box", "Start Time", "End Time", or "MSN". And then stop once all the metadata types have been collected

In [9]:
# This makes a nested dictionary of file paths to each individual metadata type
file_path_to_meta_data = extract.metadata.get_all_med_pc_meta_data_from_files(list_of_files=all_med_pc_file)

In [10]:
for key, value in file_path_to_meta_data.items():
    print("File path: {}".format(key))
    print("Metadata types and associated values: {}".format(value))
    break

File path: ./data/med_pc/2022-05-20_10h24m_Subject 2.1 v 1.3.txt
Metadata types and associated values: {'File': 'C:\\MED-PC\\Data\\2022-05-20_10h24m_Subject 2.1 v 1.3.txt', 'Start Date': '05/20/22', 'End Date': '05/20/22', 'Subject': '2.1 v 1.3', 'Experiment': 'Pilot of Pilot', 'Group': 'Cage 2', 'Box': '4', 'Start Time': '10:24:10', 'End Time': '11:55:14', 'MSN': 'C57_reward'}


## Making a Dataframe out of the Metadata

In [11]:
# Turning the dictionary into a Pandas Dataframe
metadata_df = pd.DataFrame.from_dict(file_path_to_meta_data, orient="index")
# Resetting the index because currently the file path is the index 
metadata_df = metadata_df.reset_index()

In [12]:
metadata_df.head()

,index,File,Start Date,End Date,Subject,Experiment,Group,Box,Start Time,End Time,MSN
0,./data/med_pc/2022-05-20_10h24m_Subject 2.1 v ...,C:\MED-PC\Data\2022-05-20_10h24m_Subject 2.1 v...,05/20/22,05/20/22,2.1 v 1.3,Pilot of Pilot,Cage 2,4,10:24:10,11:55:14,C57_reward
1,./data/med_pc/2022-05-19_09h09m_Subject 2.3v1....,C:\MED-PC\Data\2022-05-19_09h09m_Subject 2.3v1...,05/19/22,05/19/22,2.3v1.4,Pilot of Pilot,Cage 1,2,09:09:40,09:51:34,C57_reward


In [13]:
metadata_df.tail()

,index,File,Start Date,End Date,Subject,Experiment,Group,Box,Start Time,End Time,MSN
0,./data/med_pc/2022-05-20_10h24m_Subject 2.1 v ...,C:\MED-PC\Data\2022-05-20_10h24m_Subject 2.1 v...,05/20/22,05/20/22,2.1 v 1.3,Pilot of Pilot,Cage 2,4,10:24:10,11:55:14,C57_reward
1,./data/med_pc/2022-05-19_09h09m_Subject 2.3v1....,C:\MED-PC\Data\2022-05-19_09h09m_Subject 2.3v1...,05/19/22,05/19/22,2.3v1.4,Pilot of Pilot,Cage 1,2,09:09:40,09:51:34,C57_reward


- Just getting the numbers out of the column that contains the cage information

## Inputting all the MED-PC log files

- Example of what the MED-PC Script looks like that was ran when recording the behaviors

In [14]:
with open("./data/med_pc/C57_reward.MPC") as f:
    lines = f.readlines()
    for line in lines[:100]:
        print(line)

\v3 stop tone with poke

\v3.2 monitor port entries AND exits



\INPUTS

^port = 8



\OUTPUTS

^fan = 16

^houselight = 11

^tone1 = 2

^tone2 = 3

^tone3 = 4

^tone4 = 5

^pump = 9

^whitenoise = 1

^laser=10

^csout = 5

^peout = 15

^cs1out = 17

^cs2out = 13

^cs3out = 14



\EXP SETTINGS

^ncsNoShock = 0

^initCS1trials = 3



\ARRAYS

DIM P = 20000 \Port entry time stamp array

DIM Q = 2500 \US delivery time stamp array (absolute)

DIM R = 2500 \US time stamp array (relative to last CS)

DIM W = 2500 \ITI values used for CS

DIM S = 2500 \CS presentation values (absolute - every time light turns on)

DIM N = 20000 \Port exit time stamp array

DIM K = 2500 \CS type

DIM B = 2500 \shock intensity



LIST V = 80", 90", 80", 75", 100", 95", 90", 80", 90", 100", 90", 120", 90", 85", 90", 95", 120", 80", 95", 80", 90", 80", 75", 100", 95", 90", 80", 90", 100", 90", 120", 90", 85", 90", 95", 90", 80", 90", 100", 90", 90", 90", 90", 90", 90", 90", 90", 90", 90", 90", 90", 90", 90", 90"

- We will be using the comments in the MED-PC script(which starts with the `\`) to create a name for the variables. By default, MED-PC uses a single letter as the name of the variable.
    - This will use the medpc2excel library found in https://github.com/cyf203/medpc2excel
- Example of the comments in the MED-PC script that we will use the names from:

```
\Variables

\A - Time since last CS

\B - Shock intensity

\C - Counter array

\D - Current ITI value

\E - CS ITI values for first few trials

\F - Shock duration

\G -

\H -

\I -

\J - Shock intensity repo

\K - CS type

\L -

\M - CS type repo

\N - Port exit time stamp array

\O -

\P - Port entry time stamp array

\Q - Sucrose delivery time stamp array (absolute)

\R - Sucrose delivery time stamp array (relative to last CS)

\S - CS presentation time stamp array

\T - Session timer

\U - Time since last CS presentation

\V - List of CS ITI values (tone + houselight)

\W - ITI values used for CS one each trial

\X -

\Y - Beam break monitor variable

\Z -
```

In [15]:
with open(all_med_pc_file[0]) as f:
    lines = f.readlines()
    for line in lines[:20]:
        print(line)

File: C:\MED-PC\Data\2022-05-20_10h24m_Subject 2.1 v 1.3.txt







Start Date: 05/20/22

End Date: 05/20/22

Subject: 2.1 v 1.3

Experiment: Pilot of Pilot

Group: Cage 2

Box: 4

Start Time: 10:24:10

End Time: 11:55:14

MSN: C57_reward

A:    1599.000

D:    8000.000

F:    2000.000

G:       0.000

H:       0.000

I:       0.000

L:       0.000



- **Please make sure that the corresponding `.mpc` file (aka the MED-PC script) that was ran to create the log file, is also in the same folder**

In [16]:
concatted_medpc_df = extract.dataframe.get_medpc_dataframe_from_list_of_files(medpc_files=all_med_pc_file)
concatted_medpc_df = concatted_medpc_df.reset_index(drop=True)

In [17]:
concatted_medpc_df.head()

,(P)Portentry,(Q)USdelivery,(R)UStime,(W)ITIvalues,(S)CSpresentation,(N)Portexit,(K)CStype,(B)shockintensity,date,subject,file_path
0,1.89,64.0,399.0,0.0,60.01,3.07,1.0,0.0,20220520,2.1 v 1.3,./data/med_pc/2022-05-20_10h24m_Subject 2.1 v ...
1,3.36,144.0,399.0,0.0,140.01,3.50,1.0,0.0,20220520,2.1 v 1.3,./data/med_pc/2022-05-20_10h24m_Subject 2.1 v ...
2,4.99,234.0,399.0,0.0,230.01,6.80,1.0,0.0,20220520,2.1 v 1.3,./data/med_pc/2022-05-20_10h24m_Subject 2.1 v ...
3,7.57,314.0,399.0,0.0,310.01,7.61,1.0,0.0,20220520,2.1 v 1.3,./data/med_pc/2022-05-20_10h24m_Subject 2.1 v ...
4,8.00,389.0,399.0,0.0,385.01,8.51,1.0,0.0,20220520,2.1 v 1.3,./data/med_pc/2022-05-20_10h24m_Subject 2.1 v ...


In [18]:
concatted_medpc_df.tail()

,(P)Portentry,(Q)USdelivery,(R)UStime,(W)ITIvalues,(S)CSpresentation,(N)Portexit,(K)CStype,(B)shockintensity,date,subject,file_path
5077,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,20220519,2.3v1.4,./data/med_pc/2022-05-19_09h09m_Subject 2.3v1....
5078,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,20220519,2.3v1.4,./data/med_pc/2022-05-19_09h09m_Subject 2.3v1....
5079,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,20220519,2.3v1.4,./data/med_pc/2022-05-19_09h09m_Subject 2.3v1....
5080,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,20220519,2.3v1.4,./data/med_pc/2022-05-19_09h09m_Subject 2.3v1....
5081,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,20220519,2.3v1.4,./data/med_pc/2022-05-19_09h09m_Subject 2.3v1....


In [19]:
concatted_medpc_df = concatted_medpc_df[concatted_medpc_df["(S)CSpresentation"] % 1000 != 0].dropna()

- Dropping all rows that don't have the CS presentation time

In [20]:
cs_time_df = concatted_medpc_df[concatted_medpc_df["(P)Portentry"] != 0.0].dropna(subset="(P)Portentry")

In [21]:
cs_time_df["file_path"].unique()

array(['./data/med_pc/2022-05-20_10h24m_Subject 2.1 v 1.3.txt',
       './data/med_pc/2022-05-19_09h09m_Subject 2.3v1.4.txt'],
      dtype=object)

# Adding the time of the start of the video

In [22]:
from datetime import datetime, timedelta


In [23]:

cage_1_video_start_time = datetime(2022, 5, 19, 9, 20, 28)
cage_2_video_start_time = datetime(2022, 5, 20, 11, 24, 29)

In [24]:
cage_1_video_start_time

datetime.datetime(2022, 5, 19, 9, 20, 28)

In [25]:
cage_2_video_start_time

datetime.datetime(2022, 5, 20, 11, 24, 29)

In [26]:
cage_1_video_start_time + timedelta(seconds=60)

datetime.datetime(2022, 5, 19, 9, 21, 28)

## Correcting for the delay in recording time

- In cage 1 video, tone 2(140 seconds) happened at 2 minutes 31 seconds(151 seconds)
    - In theory 11 second delay between the video time suggested by MED-PC versus the actual time in the video. 

- In cage 2 video, tone 2(140 seconds) happened at 2 minutes 29 seconds(149 seconds)
    - In theory 9 second delay between the video time suggested by MED-PC versus the actual time in the video. 

In [27]:
cage_1_video_delay_duration = 11
cage_2_video_delay_duration = 9


## Adding a column for the tone/port entry time in the video

In [28]:
concatted_medpc_df["file_path"].unique()

array(['./data/med_pc/2022-05-20_10h24m_Subject 2.1 v 1.3.txt',
       './data/med_pc/2022-05-19_09h09m_Subject 2.3v1.4.txt'],
      dtype=object)

In [29]:
cage_2_medpc_df = concatted_medpc_df[concatted_medpc_df["file_path"] == concatted_medpc_df["file_path"].unique()[0]]

In [30]:
cage_1_medpc_df = concatted_medpc_df[concatted_medpc_df["file_path"] == concatted_medpc_df["file_path"].unique()[1]]

In [31]:
cage_1_medpc_df.head()

,(P)Portentry,(Q)USdelivery,(R)UStime,(W)ITIvalues,(S)CSpresentation,(N)Portexit,(K)CStype,(B)shockintensity,date,subject,file_path
2541,329.87,64.0,399.0,0.0,60.01,332.03,1.0,0.0,20220519,2.3v1.4,./data/med_pc/2022-05-19_09h09m_Subject 2.3v1....
2542,356.64,389.0,399.0,0.0,140.01,356.68,1.0,0.0,20220519,2.3v1.4,./data/med_pc/2022-05-19_09h09m_Subject 2.3v1....
2543,382.57,489.0,399.0,0.0,230.01,383.32,1.0,0.0,20220519,2.3v1.4,./data/med_pc/2022-05-19_09h09m_Subject 2.3v1....
2544,389.84,584.0,399.0,0.0,310.01,410.48,1.0,0.0,20220519,2.3v1.4,./data/med_pc/2022-05-19_09h09m_Subject 2.3v1....
2545,410.75,674.0,399.0,0.0,385.01,411.59,1.0,0.0,20220519,2.3v1.4,./data/med_pc/2022-05-19_09h09m_Subject 2.3v1....


In [32]:
cage_2_medpc_df.head()

,(P)Portentry,(Q)USdelivery,(R)UStime,(W)ITIvalues,(S)CSpresentation,(N)Portexit,(K)CStype,(B)shockintensity,date,subject,file_path
0,1.89,64.0,399.0,0.0,60.01,3.07,1.0,0.0,20220520,2.1 v 1.3,./data/med_pc/2022-05-20_10h24m_Subject 2.1 v ...
1,3.36,144.0,399.0,0.0,140.01,3.50,1.0,0.0,20220520,2.1 v 1.3,./data/med_pc/2022-05-20_10h24m_Subject 2.1 v ...
2,4.99,234.0,399.0,0.0,230.01,6.80,1.0,0.0,20220520,2.1 v 1.3,./data/med_pc/2022-05-20_10h24m_Subject 2.1 v ...
3,7.57,314.0,399.0,0.0,310.01,7.61,1.0,0.0,20220520,2.1 v 1.3,./data/med_pc/2022-05-20_10h24m_Subject 2.1 v ...
4,8.00,389.0,399.0,0.0,385.01,8.51,1.0,0.0,20220520,2.1 v 1.3,./data/med_pc/2022-05-20_10h24m_Subject 2.1 v ...


In [33]:
cage_1_medpc_df["tone_playing_time_in_video"] = cage_1_medpc_df["(S)CSpresentation"] + cage_1_video_delay_duration
cage_1_medpc_df["port_entry_time_in_video"] = cage_1_medpc_df["(P)Portentry"] + cage_1_video_delay_duration


In [34]:
cage_2_medpc_df["tone_playing_time_in_video"] = cage_2_medpc_df["(S)CSpresentation"] + cage_2_video_delay_duration
cage_2_medpc_df["port_entry_time_in_video"] = cage_2_medpc_df["(P)Portentry"] + cage_2_video_delay_duration


In [35]:
cage_1_medpc_df.head()

,(P)Portentry,(Q)USdelivery,(R)UStime,(W)ITIvalues,(S)CSpresentation,(N)Portexit,(K)CStype,(B)shockintensity,date,subject,file_path,tone_playing_time_in_video,port_entry_time_in_video
2541,329.87,64.0,399.0,0.0,60.01,332.03,1.0,0.0,20220519,2.3v1.4,./data/med_pc/2022-05-19_09h09m_Subject 2.3v1....,71.01,340.87
2542,356.64,389.0,399.0,0.0,140.01,356.68,1.0,0.0,20220519,2.3v1.4,./data/med_pc/2022-05-19_09h09m_Subject 2.3v1....,151.01,367.64
2543,382.57,489.0,399.0,0.0,230.01,383.32,1.0,0.0,20220519,2.3v1.4,./data/med_pc/2022-05-19_09h09m_Subject 2.3v1....,241.01,393.57
2544,389.84,584.0,399.0,0.0,310.01,410.48,1.0,0.0,20220519,2.3v1.4,./data/med_pc/2022-05-19_09h09m_Subject 2.3v1....,321.01,400.84
2545,410.75,674.0,399.0,0.0,385.01,411.59,1.0,0.0,20220519,2.3v1.4,./data/med_pc/2022-05-19_09h09m_Subject 2.3v1....,396.01,421.75


In [36]:
cage_1_medpc_df.head(n=50)

,(P)Portentry,(Q)USdelivery,(R)UStime,(W)ITIvalues,(S)CSpresentation,(N)Portexit,(K)CStype,(B)shockintensity,date,subject,file_path,tone_playing_time_in_video,port_entry_time_in_video
2541,329.87,64.0,399.0,0.0,60.01,332.03,1.0,0.0,20220519,2.3v1.4,./data/med_pc/2022-05-19_09h09m_Subject 2.3v1....,71.01,340.87
2542,356.64,389.0,399.0,0.0,140.01,356.68,1.0,0.0,20220519,2.3v1.4,./data/med_pc/2022-05-19_09h09m_Subject 2.3v1....,151.01,367.64
2543,382.57,489.0,399.0,0.0,230.01,383.32,1.0,0.0,20220519,2.3v1.4,./data/med_pc/2022-05-19_09h09m_Subject 2.3v1....,241.01,393.57
2544,389.84,584.0,399.0,0.0,310.01,410.48,1.0,0.0,20220519,2.3v1.4,./data/med_pc/2022-05-19_09h09m_Subject 2.3v1....,321.01,400.84
2545,410.75,674.0,399.0,0.0,385.01,411.59,1.0,0.0,20220519,2.3v1.4,./data/med_pc/2022-05-19_09h09m_Subject 2.3v1....,396.01,421.75
2546,411.71,754.0,399.0,0.0,485.01,412.80,1.0,0.0,20220519,2.3v1.4,./data/med_pc/2022-05-19_09h09m_Subject 2.3v1....,496.01,422.71
2547,415.91,844.0,399.0,0.0,580.01,416.03,1.0,0.0,20220519,2.3v1.4,./data/med_pc/2022-05-19_09h09m_Subject 2.3v1....,591.01,426.91
2548,416.08,944.0,399.0,0.0,670.01,421.43,1.0,0.0,20220519,2.3v1.4,./data/med_pc/2022-05-19_09h09m_Subject 2.3v1....,681.01,427.08
2549,422.09,1034.0,399.0,0.0,750.01,422.21,1.0,0.0,20220519,2.3v1.4,./data/med_pc/2022-05-19_09h09m_Subject 2.3v1....,761.01,433.09
2550,422.41,1154.0,399.0,0.0,840.01,423.68,1.0,0.0,20220519,2.3v1.4,./data/med_pc/2022-05-19_09h09m_Subject 2.3v1....,851.01,433.41


In [37]:
cage_1_medpc_df.to_csv("./data/proc/cage_1_medpc.csv")

In [38]:
cage_2_medpc_df.to_csv("./data/proc/cage_2_medpc.csv")

In [39]:
raise ValueError("Rest of the notebook is previous work")

ValueError: Rest of the notebook is previous work

# Stuff Below if previous work

# Making a column for the Controlle Stimulus time

- Because the hour, the minute, and the seconds are all in different columns we will combine them into one

In [ ]:
cs_time_df.columns

- Making the numbers for hours, minutes, and seconds into whole numbers

In [ ]:
cs_time_df["cs_second"] = cs_time_df["(G)controlled_stimulus_secondscomputer"].astype(int).astype(str)
cs_time_df["cs_minute"] = cs_time_df["(H)controlled_stimulus_minutescomputer"].astype(int).astype(str)
cs_time_df["cs_hour"] = cs_time_df["(I)controlled_stimulus_hourscomputer"].astype(int).astype(str)


- Combining the hours, minutes, and seconds into one column as a string

In [ ]:
cs_time_df["cs_time_str"] = cs_time_df.apply(lambda x: ":".join([x["cs_hour"], x["cs_minute"],  x["cs_second"]]), axis=1)

- Converting the time into a Python datetime object
    - This will allow us to make calculations

In [ ]:
cs_time_df["cs_time_object"] = cs_time_df["cs_time_str"].apply(lambda x: datetime.strptime(x, '%H:%M:%S'))


In [ ]:
cs_time_df

# Making a column for the Port Entry time

- Because the hour, the minute, and the seconds are all in different columns we will combine them into one

In [ ]:
cs_time_df.columns

- Making the numbers for hours, minutes, and seconds into whole numbers

In [ ]:
cs_time_df["pe_second"] = cs_time_df["(B)port_entry_secondscomputer"].astype(int).astype(str)
cs_time_df["pe_minute"] = cs_time_df["(F)port_entry_minutescomputer"].astype(int).astype(str)
cs_time_df["pe_hour"] = cs_time_df["(J)port_entry_hourscomputer"].astype(int).astype(str)


- Combining the hours, minutes, and seconds into one column as a string

In [ ]:
cs_time_df["pe_time_str"] = cs_time_df.apply(lambda x: ":".join([x["pe_hour"], x["pe_minute"],  x["pe_second"]]), axis=1)

- Converting the time into a Python datetime object
    - This will allow us to make calculations

In [ ]:
cs_time_df["pe_time_object"] = cs_time_df["pe_time_str"].apply(lambda x: datetime.strptime(x, '%H:%M:%S'))


In [ ]:
cs_time_df["pe_time_object"]

In [ ]:
cs_time_df["pe_time_object"]

In [ ]:
cs_time_df

# Importing Video Data

- These CSVs are create in Bonsai when the video is being recorded. They give a time stamp of when the video is first being recorded.

In [ ]:
video_recording_time_stamps = pd.read_csv("./data/automatic_30_min_end_test/clock_test_time_2022-07-13T20_02_37.csv", header=None)

In [ ]:
video_recording_time_stamps.head()

- The video is first being recorded in the time specified by the cell in the first row and the first column
    - We will try to get on the hour, minute, seconds from this
        - By getting everthing after the space, and then removing everything before the period

In [ ]:
video_recording_time_stamps["time"] = video_recording_time_stamps[1].apply(lambda x: x.split()[-1].split(".")[0])


In [ ]:
video_recording_time_stamps.head()

- Getting the first time that's listed, which is when the video starts being recorded

In [ ]:
video_recording_start_time_str = video_recording_time_stamps["time"][0]


In [ ]:
video_recording_start_time_str

- Converting this into a datetime object so that we can make calculations

In [ ]:
video_recording_start_time_object = datetime.strptime(video_recording_start_time_str, '%H:%M:%S')


In [ ]:
video_recording_start_time_object

# Syncing the MED-PC timestamps with the time in the video

- To get when in the video something happens(i.e. 10 seconds), we will calculate the difference of the time of the event with the time that the video first starts being recorded
- NOTE: Make sure that you are using the associated files for the MED-PC recording file and the video file

In [ ]:
first_medpc_recording_file = cs_time_df["file_path"].unique()[0]

first_cs_time_df = cs_time_df[cs_time_df["file_path"] == first_medpc_recording_file]

In [ ]:
# The time of the first controlled stimulus 
first_cs_time_df["cs_time_object"].iloc[0]

In [ ]:
# The time of the second controlled stiumuls
first_cs_time_df["cs_time_object"].iloc[1]

In [ ]:
# The time that the video started being recorded
video_recording_start_time_object

In [ ]:
# The time in the video that the controlled stimulus occurs
cs_01_time_object = first_cs_time_df["cs_time_object"].iloc[0] - video_recording_start_time_object 
cs_02_time_object = first_cs_time_df["cs_time_object"].iloc[1] - video_recording_start_time_object 
cs_03_time_object = first_cs_time_df["cs_time_object"].iloc[2] - video_recording_start_time_object 

In [ ]:
cs_01_time_object.seconds

In [ ]:
cs_02_time_object.seconds

In [ ]:
cs_03_time_object.seconds

# Syncing the MED-PC Port Entires with the time in the video

- To get when in the video something happens(i.e. 10 seconds), we will calculate the difference of the time of the event with the time that the video first starts being recorded
- NOTE: Make sure that you are using the associated files for the MED-PC recording file and the video file

In [ ]:
first_cs_time_df["(F)port_entry_minutescomputer"].head(n=50)

In [ ]:
first_cs_time_df["(F)port_entry_minutescomputer"].tail(n=50)

In [ ]:
first_cs_time_df["(B)port_entry_secondscomputer"].head(n=50)

In [ ]:
first_cs_time_df["(B)port_entry_secondscomputer"].tail(n=50)

In [ ]:
# The time in the video that the controlled stimulus occurs
port_entry_01_time_object = first_cs_time_df["pe_time_object"].iloc[0] - video_recording_start_time_object 
port_entry_02_time_object = first_cs_time_df["pe_time_object"].iloc[5] - video_recording_start_time_object 
port_entry_03_time_object = first_cs_time_df["pe_time_object"].iloc[8] - video_recording_start_time_object 

In [ ]:
first_cs_time_df["pe_time_object"].iloc[8]

In [ ]:
port_entry_01_time_object.seconds

In [ ]:
port_entry_02_time_object.seconds

In [ ]:
port_entry_03_time_object.seconds

# Trimming the video

In [ ]:
video_object = VideoFileClip("./data/automatic_30_min_end_test/2clock_test_recording_2022-07-13T20_02_39.avi")

In [ ]:
video_object.duration

fixed func issue with: https://github.com/Zulko/moviepy/issues/1765


In [ ]:
cs1_trimmed_video = video_object.subclip(cs_01_time_object.seconds, cs_02_time_object.seconds + 20)

In [ ]:
cs2_trimmed_video = video_object.subclip(cs_02_time_object.seconds, cs_03_time_object.seconds + 20)

In [ ]:
cs3_trimmed_video = video_object.subclip(cs_03_time_object.seconds, video_object.duration)

In [ ]:
pe1_trimmed_video = video_object.subclip(port_entry_01_time_object.seconds, port_entry_02_time_object.seconds + 20)

In [ ]:
pe2_trimmed_video = video_object.subclip(port_entry_02_time_object.seconds, port_entry_03_time_object.seconds + 20)

In [ ]:
pe3_trimmed_video = video_object.subclip(port_entry_03_time_object.seconds, video_object.duration)

In [ ]:
cs1_trimmed_video.to_videofile("./data/automatic_30_min_end_test/trimmed_cs1.mp4")

In [ ]:
cs2_trimmed_video.to_videofile("./data/automatic_30_min_end_test/trimmed_cs2.mp4")

In [ ]:
cs3_trimmed_video.to_videofile("./data/automatic_30_min_end_test//trimmed_cs3.mp4")

In [ ]:
pe1_trimmed_video = video_object.subclip(port_entry_01_time_object.seconds, port_entry_02_time_object.seconds + 20)

In [ ]:
pe2_trimmed_video = video_object.subclip(port_entry_02_time_object.seconds, port_entry_03_time_object.seconds + 20)

In [ ]:
pe3_trimmed_video = video_object.subclip(port_entry_03_time_object.seconds, video_object.duration)

In [ ]:
pe1_trimmed_video.to_videofile("./data/automatic_30_min_end_test/trimmed_pe1.mp4")

In [ ]:
pe2_trimmed_video.to_videofile("./data/automatic_30_min_end_test/trimmed_pe2.mp4")

In [ ]:
pe3_trimmed_video.to_videofile("./data/automatic_30_min_end_test//trimmed_pe3.mp4")

In [ ]:
Video("./data/automatic_30_min_end_test/trimmed_cs1.mp4")